Data was retrieved from the oec data site: https://atlas.media.mit.edu/en/resources/data/

In [1]:
import pandas as pd
from IPython.display import display

In [2]:
!ls

country_names.tsv	  year_origin_destination_hs07_4.tsv
pandas_filter_data.ipynb  year_origin_destination_hs96_4.tsv
products_hs_07.tsv	  year_origin_hs96_4.tsv
products_hs_96.tsv


Get the country names and products table

In [3]:
country_names = pd.read_table('country_names.tsv')
products = pd.read_table('products_hs_07.tsv')
products

,id,hs07,name
0,10101,101,"Live horses, asses, mules and hinnies."
1,1010110,10110,Live horses/asses/mules/hinnies: pure-bred bre...
2,1010190,10190,Live horses/asses/mules/hinnies other than pur...
3,10102,102,Live bovine animals.
4,1010210,10210,Live bovine animals: pure-bred breeding animals
5,1010290,10290,Live bovine animals other than pure-bred breed...
6,10103,103,Live swine.
7,1010310,10310,Live swine: pure-bred breeding animals
8,1010391,10391,Live swine other than pure-bred breeding anima...
9,1010392,10392,Live swine other than pure-bred breeding anima...


For example lets try to get some of the tea traffic

In [4]:
country_names.head()

,id,id_3char,name
0,afago,ago,Angola
1,afbdi,bdi,Burundi
2,afben,ben,Benin
3,afbfa,bfa,Burkina Faso
4,afbwa,bwa,Botswana


In [5]:
def filter_chunks(iterator_frame, condition, max_chunks=None, chunksize=10000):
    chunk = next(iterator_frame)
    result_df = chunk[eval(condition)]#Generalized conditions
    cur_chunk = 1
    for chunk in iterator_frame:
        if max_chunks is not None:
            if cur_chunk >= max_chunks:
                break
        filtered_chunk = chunk[eval(condition)]
        result_df = pd.concat([result_df, filtered_chunk])
        cur_chunk += 1
    return result_df

In [6]:
route_table = pd.read_table('year_origin_destination_hs07_4.tsv', chunksize=10000)
condition = "(chunk['origin'] == 'ken')"#Get all of the kenyan exports
my_df = filter_chunks(route_table, condition)

In [7]:
export_length = my_df[(my_df['export_val'].notnull())].shape[0]
import_length = my_df[(my_df['import_val'].notnull())].shape[0]
orig_length = my_df.shape[0]
print("Original Length: {}, Export Length: {}, Import Length: {}, Sum: {}".format(orig_length, export_length, import_length, import_length + export_length))

Original Length: 147239, Export Length: 58091, Import Length: 106860, Sum: 164951


In [8]:
#Lets map the origin id's to the origin country
kenya_transactions_with_countries = pd.merge(my_df,country_names,left_on='origin', right_on='id_3char')
del kenya_transactions_with_countries['id_3char']
del kenya_transactions_with_countries['id']
kenya_transactions_with_countries.rename(index=str, columns={"name":"from_name"}, inplace=True)
#Now do the same thing for the destination country
kenya_transactions_with_countries = pd.merge(kenya_transactions_with_countries,country_names,left_on='dest', right_on='id_3char')
del kenya_transactions_with_countries['id_3char']
del kenya_transactions_with_countries['id']
kenya_transactions_with_countries.rename(index=str, columns={"name":"to_name"}, inplace=True)
#Show me 25 random records
display(kenya_transactions_with_countries.sample(25))

,year,origin,dest,hs07,export_val,import_val,from_name,to_name
2649,2008,ken,cod,8701,51758.00,NaN,Kenya,Democratic Republic of the Congo
59442,2011,ken,ind,8512,1786.00,144550.00,Kenya,India
3888,2013,ken,cod,9609,44188.00,NaN,Kenya,Democratic Republic of the Congo
88721,2010,ken,blx,9011,NaN,2812.00,Kenya,Belgium-Luxembourg
7425,2009,ken,eth,7615,1024020.71,NaN,Kenya,Ethiopia
85546,2010,ken,aut,3105,NaN,2729.97,Kenya,Austria
59935,2012,ken,ind,5107,NaN,19656.00,Kenya,India
75475,2014,ken,pak,8483,NaN,34607.00,Kenya,Pakistan
135834,2008,ken,usa,8480,90832.00,59450.98,Kenya,United States
134494,2012,ken,gtm,8528,1011.00,NaN,Kenya,Guatemala


In [9]:
kenya_trans_with_products = pd.merge(kenya_transactions_with_countries, products, left_on='hs07', right_on='hs07')
del kenya_trans_with_products['id']#Delete foreign key 
kenya_trans_with_products.rename(index=str, columns={"name":"product_name"}, inplace=True) # Rename column
kenya_trans_with_products[(kenya_trans_with_products['year'] == 2010) & (kenya_trans_with_products['dest']=='gbr')].sample(50)

,year,origin,dest,hs07,export_val,import_val,from_name,to_name,product_name
141278,2010,ken,gbr,5909,NaN,11557.89,Kenya,United Kingdom,"Textile hosepiping and similar textile tubing,..."
17256,2010,ken,gbr,4202,204330.99,59675.82,Kenya,United Kingdom,"Trunks, suit-cases, vanity-cases, executive-ca..."
49090,2010,ken,gbr,7307,NaN,1326923.91,Kenya,United Kingdom,"Tube or pipe fittings (for example, couplings,..."
47368,2010,ken,gbr,7215,NaN,21916.02,Kenya,United Kingdom,Other bars and rods of iron or non-alloy steel.
54250,2010,ken,gbr,8409,58359.00,3693723.94,Kenya,United Kingdom,Parts suitable for use solely or principally w...
31252,2010,ken,gbr,3208,NaN,317794.16,Kenya,United Kingdom,Paints and varnishes (including enamels and la...
50261,2010,ken,gbr,7315,NaN,654577.28,Kenya,United Kingdom,"Chain and parts thereof, of iron or steel."
108996,2010,ken,gbr,5204,NaN,16894.96,Kenya,United Kingdom,"Cotton sewing thread, whether or not put up fo..."
47540,2010,ken,gbr,7216,NaN,305733.49,Kenya,United Kingdom,"Angles, shapes and sections of iron or non-all..."
59878,2010,ken,gbr,8502,NaN,20031374.99,Kenya,United Kingdom,Electric generating sets and rotary converters.


In [10]:
exports = kenya_trans_with_products[(kenya_trans_with_products['export_val'].notnull())]
exports['export_val'].sum()

38590981300.73